In [2]:
# First we want to examine the data
# For this, let's create a module so we can read the SQLite contents

import sqlite3


class SqlLiteExecutor(object):

    def __init__(self, db_file):
        self.db_file = db_file
        self.conn = None

    def __enter__(self) -> sqlite3.Cursor:
        self.conn = sqlite3.connect(self.db_file)
        self.conn.row_factory = sqlite3.Row
        return self.conn.cursor()

    def __exit__(self ,type, value, traceback):
        if self.conn:
            self.conn.close()
        return None

In [3]:
# Now, let's look at the first data objective
# The goal is to "calculate number of unique customers of each gender in these age groups"
#
# In order to achieve this, we can do the following:
# 1. Create a list of unique customer (using data from both tables)
# 2. Segment these customers into 10 age groups
# 3. Flag each customer with the correct `age_segment` that he is in
# 3. Calculate number of unique customers of each gender in each age group

unique_customer_list_sql = open("./sql/unique_customer_list.sql", "r").read()
db_path = r"./db/assessment_database.sqlite"

``` mysql
-- Get unique ages that exist within the dataset
WITH unique_ages AS (
  SELECT DISTINCT
    Age AS age
  FROM
    CustomerList1
  UNION
  SELECT DISTINCT
    Age AS age
  FROM
    CustomerList2
),
-- Segment these ages based on intervals
unique_ages_segmented AS (
  SELECT
    age AS segment_age,
    CASE
      WHEN age >=0 AND age < 18 THEN '0-18'
      WHEN age >=18 AND age < 25 THEN '18-25'
      WHEN age >=25 AND age < 30 THEN '25-30'
      WHEN age >=30 AND age < 35 THEN '30-35'
      WHEN age >=35 AND age < 40 THEN '35-40'
      WHEN age >=40 AND age < 50 THEN '40-50'
      WHEN age >=50 AND age < 60 THEN '50-60'
      WHEN age >=60 AND age < 70 THEN '60-70'
      WHEN age >=70 AND age < 80 THEN '70-80'
      WHEN age >=80 THEN '80+'
    END AS age_segment
  FROM
    unique_ages
),
-- Join the age segments to the Customers Table, adding an `age_segment` flag
unique_customers_grouped AS (
  SELECT
    CAST(cl1.Customer_id AS INT)   AS customer_id,
    CAST(cl1.Age AS INT)           AS age,
    IFNULL(Gender, 'unknown')      AS gender,
    IFNULL(seg.age_segment, 'N/A') AS age_segment
  FROM
    CustomerList1 cl1
  LEFT JOIN unique_ages_segmented seg ON seg.segment_age = cl1.Age
  WHERE
    customer_id IS NOT NULL
  UNION
  SELECT
    CAST(cl2.Customer_id AS INT)   AS customer_id,
    CAST(cl2.Age AS INT)           AS age,
    IFNULL(Gender, 'unknown')      AS gender,
    IFNULL(seg.age_segment, 'N/A') AS age_segment
  FROM
   CustomerList2 cl2
  LEFT JOIN unique_ages_segmented seg ON seg.segment_age = cl2.Age
  WHERE
    customer_id IS NOT NULL
)
-- Count the number of customer_ids grouped by `age_segment` and `gender`
SELECT
  age_segment                 AS age_segment,
  gender                      AS gender,
  COUNT(DISTINCT customer_id) AS customer_count
FROM
  unique_customers_grouped
GROUP BY 1, 2
;
```

In [4]:
# Read the database and run the SQL script

from typing import Iterable

with SqlLiteExecutor(db_path) as conn:
    conn.execute(unique_customer_list_sql)

    rows: Iterable[sqlite3.Row] = conn.fetchall()
    for row in rows:
        print(dict(row))

{'age_segment': '0-18', 'gender': 'male', 'customer_count': 2}
{'age_segment': '18-25', 'gender': 'female', 'customer_count': 9}
{'age_segment': '18-25', 'gender': 'male', 'customer_count': 9}
{'age_segment': '25-30', 'gender': 'female', 'customer_count': 3}
{'age_segment': '25-30', 'gender': 'male', 'customer_count': 15}
{'age_segment': '30-35', 'gender': 'female', 'customer_count': 17}
{'age_segment': '30-35', 'gender': 'male', 'customer_count': 22}
{'age_segment': '35-40', 'gender': 'female', 'customer_count': 21}
{'age_segment': '35-40', 'gender': 'male', 'customer_count': 35}
{'age_segment': '40-50', 'gender': 'female', 'customer_count': 90}
{'age_segment': '40-50', 'gender': 'male', 'customer_count': 123}
{'age_segment': '40-50', 'gender': 'unknown', 'customer_count': 3}
{'age_segment': '50-60', 'gender': 'female', 'customer_count': 145}
{'age_segment': '50-60', 'gender': 'male', 'customer_count': 216}
{'age_segment': '50-60', 'gender': 'unknown', 'customer_count': 5}
{'age_segme

In [5]:
# Then comes the second question:
# What is the most profitable customer group for the store?
# For this we again need the age ranges
# We however also need the number of items sold and the values from other tables

orders_sql = open("./sql/orders.sql", "r").read()

```mysql
-- We again get the unique ages
WITH unique_ages AS (
  SELECT DISTINCT
    Age AS age
  FROM
    CustomerList1
  UNION
  SELECT DISTINCT
    Age AS age
  FROM
    CustomerList2
),
-- We then again segment them into age ranges
unique_ages_segmented AS (
  SELECT
    age AS segment_age,
    CASE
      WHEN age >=0 AND age < 18 THEN '0-18'
      WHEN age >=18 AND age < 25 THEN '18-25'
      WHEN age >=25 AND age < 30 THEN '25-30'
      WHEN age >=30 AND age < 35 THEN '30-35'
      WHEN age >=35 AND age < 40 THEN '35-40'
      WHEN age >=40 AND age < 50 THEN '40-50'
      WHEN age >=50 AND age < 60 THEN '50-60'
      WHEN age >=60 AND age < 70 THEN '60-70'
      WHEN age >=70 AND age < 80 THEN '70-80'
      WHEN age >=80 THEN '80+'
    END AS age_segment
  FROM
    unique_ages
),
-- We get all remaining columns that are useful
unique_customers_grouped AS (
  SELECT
    CAST(cl1.Customer_id AS INT)                   AS customer_id,
    CAST(cl1.Age AS INT)                           AS age,
    IFNULL(Gender, 'unknown')                      AS gender,
    country                                        AS country,
    IFNULL(ua.age_segment, 'N/A')                  AS age_segment,
    RTRIM(Name || ' ' || IFNULL(Surname, ''), ' ') AS full_name
  FROM
    CustomerList1 cl1
  LEFT JOIN unique_ages_segmented ua ON ua.segment_age = cl1.Age
  WHERE
    customer_id IS NOT NULL
  UNION
  SELECT
    CAST(cl2.Customer_id AS INT)                   AS customer_id,
    CAST(cl2.Age AS INT)                           AS age,
    IFNULL(Gender, 'unknown')                      AS gender,
    country                                        AS country,
    IFNULL(ua.age_segment, 'N/A')                  AS age_segment,
    RTRIM(Name || ' ' || IFNULL(Surname, ''), ' ') AS full_name
  FROM
   CustomerList2 cl2
  LEFT JOIN unique_ages_segmented ua ON ua.segment_age = cl2.Age
  WHERE
    customer_id IS NOT NULL
-- We add all of the necessary JOINS
SELECT
  c.customer_id                                    AS customer_id,
  c.full_name                                      AS full_name,
  c.age                                            AS age,
  c.gender                                         AS gender,
  c.country                                        AS country,
  c.age_segment                                    AS age_segment,
  o.'Order ID'                                     AS order_id,
  b.'Product Name'                                 AS product_name,
  CAST(b.'Discount per product' AS FLOAT)          AS discount_per_product,
  CASE
    WHEN IFNULL(CAST(b.'Discount per product' AS FLOAT), 0) = 0 THEN 'no_discount'
    WHEN 5 > CAST(b.'Discount per product' AS FLOAT)> 0 THEN '0-5'
    WHEN 10 > CAST(b.'Discount per product' AS FLOAT)>= 5 THEN '5-10'
    WHEN 15 > CAST(b.'Discount per product' AS FLOAT) >= 10 THEN '10-15'
    WHEN 20 > CAST(b.'Discount per product' AS FLOAT) >= 15 THEN '15-20'
    ELSE '20+'
  END AS discount_category,
  CAST(b.'Total revenue before discount' AS FLOAT) AS total_revenue_before_discount,
  CAST(b.'Total quantity' AS FLOAT)                AS quantity,
  b.Category                                       AS category
FROM
  unique_customers_grouped c
LEFT JOIN ListOfOrders o ON c.full_name = o.'Customer Name'
LEFT JOIN OrderBreakdown b ON b.'Order ID' = o.'Order ID'
;
```

In [6]:
# We then run the SQL script and output the result into a JSON lines file

from typing import Iterable
import json


orders_output_file = "./output/orders_categorized.jsonl"

with SqlLiteExecutor(r"./db/assessment_database.sqlite") as conn:
    conn.execute(orders_sql)

    rows: Iterable[sqlite3.Row] = conn.fetchall()
    with open(orders_output_file, "w+") as f:
        for row in rows:
            f.write(f"{json.dumps(dict(row))}\n")

In [7]:
# We can examine this file and check it looks ok

with open(orders_output_file, "r") as f:
    
    for _ in range(5):
        print(next(f))

{"customer_id": 100269619, "full_name": null, "age": 50, "gender": "male", "country": "Italy", "age_segment": "50-60", "order_id": null, "product_name": null, "discount_per_product": null, "discount_category": "no_discount", "total_revenue_before_discount": null, "quantity": null, "category": null}

{"customer_id": 100269619, "full_name": "Ruben Dartt", "age": 50, "gender": "male", "country": "Italy", "age_segment": "50-60", "order_id": "ES-2013-2435824", "product_name": "Advantus Stacking Tray, Ergonomic", "discount_per_product": 0.0, "discount_category": "no_discount", "total_revenue_before_discount": 23.0, "quantity": 1.0, "category": "Furniture"}

{"customer_id": 100963069, "full_name": "Mick Hernandez", "age": 55, "gender": "male", "country": "Italy", "age_segment": "50-60", "order_id": "ES-2011-5338028", "product_name": "Avery 3-Hole Punch, Clear", "discount_per_product": 0.0, "discount_category": "no_discount", "total_revenue_before_discount": 137.0, "quantity": 5.0, "category":

Now what we can do is visualise this data. For this we can use Tableau.

Because we have grouped the metrics, we can examine the `Total Revenue` per `Age Segment` category in a graph.

![revenue-by-age-segment](./screenshots/image_1.png)


We can then drill deeper and see if gender plays a role in the distribution of sales.

![revenue-by-age-segment-and-gender](./screenshots/image_2.png)

We can see that there are more males that buy products than females.

![revenue-by-gender](./screenshots/image_3.png)


So who are these males, can we make the segment even more specific?

We can do this by checking for category as well to identify the specific segments.

![revenue-by-category](./screenshots/image_4.png)

The most profitable customer group is males between the age of 50-60 that are interested 
in office supplies and technology.

Important side-note: there was a `discount` field in the data, but it is unclear what it meant (e.g. percentage of revenue, absolute value of the product, etc). Because of this it has not been used.